<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==3.0.6
!pip install -U spacy_pkuseg
!python -m spacy download zh_core_web_lg
# !wget https://github.com/explosion/spacy-models/releases/download/zh_core_web_md-3.1.0/zh_core_web_md-3.1.0.tar.gz
# !tar xvfz zh_core_web_md-3.1.0.tar.gz
# !python -m spacy download zh_core_web_trf

     |████████████████████████████████| 12.8 MB 71 kB/s 
     |████████████████████████████████| 621 kB 45.2 MB/s 
     |████████████████████████████████| 456 kB 46.0 MB/s 
     |████████████████████████████████| 9.1 MB 18.3 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 2.4 MB 5.0 MB/s 
2021-08-12 00:27:45.756792: I tensorflow/stream_

In [7]:
import spacy
from spacy import displacy

# spacyModel='zh_core_web_md-3.1.0/zh_core_web_md/zh_core_web_md-3.1.0'
nlp = spacy.load('zh_core_web_lg')

doc = nlp('恆指漲3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。')
displacy.render(doc,jupyter=True,style="ent")

# for ent in doc.ents:
#   print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [8]:
import random
from spacy.training.example import Example

TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN')         
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),(17,19,'STOCK')      
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(20,24,'DATE'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '匯控低開4%。港股收市跌。',
    {
        'entities':
        [
         (0,2,'STOCK'),(4,6,'PERCENT'),(7,9,'FIN')
        ]
    }
  ),
  (
    '港股高開173點。',
   {
       'entities': [(0,2,'FIN'),(2,4,'FIN')]
   }
  )
]

ner = nlp.get_pipe('ner')

ner.add_label('STOCK')
ner.add_label('FIN')

epoch = 50
optimizer = nlp.resume_training()

# get names of other pipes to disable them during training
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
print(f'Pipes: {other_pipes}')
with nlp.disable_pipes(*other_pipes):  # only train NER
  for itn in range(epoch):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = spacy.util.minibatch(TRAIN_DATA,size=2)
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,drop=0.2,sgd=optimizer,losses=losses)
    print("Losses", losses)

nlp.to_disk('saved_model')

print(f'NER labels: {ner.labels}')

Pipes: ['tok2vec', 'tagger', 'parser', 'attribute_ruler']
Losses {'ner': 55.15148657934887}
Losses {'ner': 40.86199329430383}
Losses {'ner': 36.56715226644883}
Losses {'ner': 34.257693074061535}
Losses {'ner': 26.166967763398134}
Losses {'ner': 23.268713507421126}
Losses {'ner': 19.78345212131535}
Losses {'ner': 16.458384596524283}
Losses {'ner': 14.502569249307788}
Losses {'ner': 11.03642824527027}
Losses {'ner': 57.438667907577}
Losses {'ner': 37.91841900826165}
Losses {'ner': 12.706588924338444}
Losses {'ner': 12.159097247724484}
Losses {'ner': 10.48908131182337}
Losses {'ner': 8.022490055788673}
Losses {'ner': 4.37153653016521}
Losses {'ner': 1.2089859231181064}
Losses {'ner': 0.6213618572232605}
Losses {'ner': 0.08414863586745633}
Losses {'ner': 0.1951008073339098}
Losses {'ner': 0.0038845426811748023}
Losses {'ner': 0.06804940456133056}
Losses {'ner': 0.002309522244715238}
Losses {'ner': 0.0026810950687636923}
Losses {'ner': 0.00037491397427491817}
Losses {'ner': 0.00012775726567

In [4]:
# for text, annot in TRAIN_DATA:
#     print(text,annot)
#     doc = nlp.make_doc(text)
#     biluo = spacy.training.offsets_to_biluo_tags(doc, annot["entities"])
#     print([t.text for t in doc], biluo)

In [12]:
# Let's give our trained model a try!

texts = [
  '恆指上升3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。',
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累，匯控、和記跌超過2%。',
  '匯控下跌2%。',
  '日本、新加坡、澳洲股市單日暴升961.22點。'
]

nlp_updated = spacy.load('saved_model')
# nlp_updated.tokenizer.pkuseg_update_user_dict(['匯控','和黄','中電','派息','低開','高開','股份回購','每股盈利'])

for text in texts:
  doc = nlp_updated(text)
  displacy.render(doc,jupyter=True,style="ent")
